<a href="https://colab.research.google.com/github/delhian/NIR/blob/main/BERTScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/delhian/NIR/blob/main/%D0%9D%D0%98%D0%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip3 install umap-learn

In [ ]:
# !pip3 install transformers

In [ ]:
# pip install word2word

In [ ]:
# pip install qc-procrustes

In [ ]:
# translator.translate('veritas lux mea', src='la')

In [ ]:
import re
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import random
import umap.umap_ as umap
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
from bokeh.models import ColumnDataSource, LabelSet, Legend
from bokeh.plotting  import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import time
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import paired_cosine_distances, pairwise_distances, cosine_similarity, cosine_distances
# https://arxiv.org/pdf/1911.12019v1.pdf

from deep_translator import GoogleTranslator
gt = GoogleTranslator(source='ru', target='en')

pd.set_option('max_colwidth', 400)

from procrustes import orthogonal, generic, rotational

from scipy import stats

import random as rn

Loading BokehJS ...

In [ ]:
# # pairs= pd.read_excel('pairs.xlsx', names = \
# #                      ['en', 'sarn','sarn_score','kurl', 'kurl_score', 'google', 'google_score', 'sum'])
# # pairs = pairs[['en','sarn','kurl', 'google', ]]

# pairs

In [ ]:
%%time
# model_name ='bert-base-uncased'
model_name ='bert-base-multilingual-cased'
model_en = AutoModel.from_pretrained(model_name)
tokenizer_en = AutoTokenizer.from_pretrained(model_name)


model_name ='DeepPavlov/rubert-base-cased'
# model_name ='DeepPavlov/rubert-base-cased-sentence'
tokenizer_ru = AutoTokenizer.from_pretrained(model_name)
model_ru = AutoModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Wall time: 8min 15s


In [ ]:
%%time
name = './Stephen King - Needful Things.txt'
with open(name, 'r',) as file:
    text_en = file.read()
print(len(text_en))

name_ru = './Нужные вещи_Сарнов.txt'
with open(name_ru, 'r', encoding='utf-8') as file:
    text_ru_sarn = file.read()
print(len(text_ru_sarn))

name_ru = './Нужные вещи_Курляндская.txt'
with open(name_ru, 'r', encoding='windows-1251') as file:
    text_ru_kurl = file.read()
print(len(text_ru_kurl))

name_ru = './Нужные-вещи_Аракелов.txt'
with open(name_ru, 'r', encoding='utf-16') as file:
    text_ru_arak = file.read()
print(len(text_ru_arak))

name_ru = './Нужные вещи_Гугл.txt'
with open(name_ru, 'r', encoding='utf-8') as file:
    text_ru_google = file.read()
print(len(text_ru_google))

1382371
1398883
1423133
1364074
1350316
Wall time: 132 ms


In [ ]:
# pairs = []
# for ends in ['ый', 'ая', 'ое', 'ые']:
#   pairs.append(pd.read_csv('/content/drive/MyDrive/pairs_' + ends+'.csv'))
# pairs = pd.concat(pairs)
# pairs = pairs.reset_index(drop=True).drop_duplicates()
# pairs.shape

In [ ]:
# phrases_ru = pd.DataFrame([['бел', 'white'], 
#                            ['черн', 'black'],
#                            ['красн', 'red'],
#                            ['желт', 'yellow'],
#                            ['зелен', 'green'],
#                            ['голуб', 'blue'],
#                            ['син', 'blue'],
#                            ['коричнев', 'brown'],
#                            ['фиолетов','violet'],
#                            ['оранжев', 'orange'],
#                            ['розов', 'pink'],
#                            ['сер', 'gray'],['сер', 'grey']], columns = ['base', 'token_en'])


# ru_ends = ['ая', 'ее', 'ого', 'ое', 'ой','ом','ому','ою','ую','ые','ый','ым','ыми','ых']
              
# phrases_ru = pd.DataFrame([[x, x + y] for y in sorted(ru_ends) for x in phrases_ru['base'].values],\
# columns = ['base','token']).set_index('base').join(phrases_ru.set_index('base')).reset_index()

# phrases_en = pd.DataFrame([['white'], 
#                            ['black'],
#                            ['red'],
#                            ['yellow'],
#                            ['green'],
#                            ['blue'],
#                            ['brown'],
#                            ['violet'],
#                            ['orange'],
#                            ['pink'],
#                            ['gray'],
#                            ['grey']], columns = {'base'})

# phrases_en['token'] = phrases_en['base'] 
# phrases_en['en'] = phrases_en['base']

In [ ]:
%%time


# def get_empty(x):
#     try:
#         tmp = re.findall('^[a-z]+$', x)
#         return tmp[0]
#     except:
#         return ''

ru_sentences_re = '[А-Я].*?[\.!?]'
en_sentences_re = '[A-Z].*?[\.!?]'

ru_to_en_dict ={}

def f_dfmake(text,
             sentences_re,
             tokenizer,
             model,
             translate):
    emb =[]
    prog = re.compile(sentences_re, re.MULTILINE | re.DOTALL )
    sentences = re.findall(prog, text)
    for txt in tqdm(sentences):
        ru_to_en_dict_len = len(ru_to_en_dict)
        if ru_to_en_dict_len%1000 == 0:
            print(f'len(ru_to_en_dict): {ru_to_en_dict_len}')
        emb_token = []
#         txt = text[seq.start() + 1: seq.end()]
        encoded_input = tokenizer(txt, return_tensors='pt')
        sentence_embedding = model(**encoded_input)[0][0,1:-1]
        for ind, token in enumerate(tokenizer.tokenize(txt)):
            word_embedding = sentence_embedding[ind].detach().numpy()
# --normilize:
            word_embedding /= np.sqrt(np.sum(word_embedding ** 2))
            token_lower = token.lower()
            if translate:
                token_en = ru_to_en_dict.get(token_lower, '')
                if token_en =='':
                    try:
                        token_en = gt.translate(token_lower).lower()
                    except:
                        token_en = token
                    ru_to_en_dict[token_lower] = token_en
            else:
                token_en = token_lower
            emb_token.append([token, token_lower, token_en, txt , word_embedding, ])  
        emb += emb_token
    emb = pd.DataFrame(emb, columns = ['token', 'token_lower', 'token_en','txt', 'word_embedding'])
    return emb

emb_ru_google = f_dfmake (text_ru_google, ru_sentences_re, tokenizer_ru, model_ru, True)
emb_ru_sarn = f_dfmake (text_ru_sarn, ru_sentences_re, tokenizer_ru, model_ru, True)
emb_ru_kurl = f_dfmake (text_ru_kurl, ru_sentences_re, tokenizer_ru, model_ru, True)
emb_ru_arak = f_dfmake (text_ru_arak, ru_sentences_re, tokenizer_ru, model_ru, True)
emb_en = f_dfmake (text_en, en_sentences_re, tokenizer_en, model_en, False)

print(emb_ru_google.shape, emb_ru_sarn.shape,emb_ru_kurl.shape, emb_ru_arak.shape, emb_en.shape)
emb_ru_google.to_pickle('emb_ru_google.pkl')
emb_ru_sarn.to_pickle('emb_ru_sarn.pkl')
emb_ru_kurl.to_pickle('emb_ru_kurl.pkl')
emb_ru_arak.to_pickle('emb_ru_arak.pkl')
emb_en.to_pickle('emb_en.pkl')

  0%|          | 0/19178 [00:00<?, ?it/s]

len(ru_to_en_dict): 0
len(ru_to_en_dict): 4000
len(ru_to_en_dict): 9000
len(ru_to_en_dict): 10000
len(ru_to_en_dict): 14000
len(ru_to_en_dict): 18000
len(ru_to_en_dict): 19000


  0%|          | 0/19823 [00:00<?, ?it/s]

len(ru_to_en_dict): 23000
len(ru_to_en_dict): 23000
len(ru_to_en_dict): 23000
len(ru_to_en_dict): 24000
len(ru_to_en_dict): 24000
len(ru_to_en_dict): 24000
len(ru_to_en_dict): 25000
len(ru_to_en_dict): 25000
len(ru_to_en_dict): 25000
len(ru_to_en_dict): 25000
len(ru_to_en_dict): 25000
len(ru_to_en_dict): 25000
len(ru_to_en_dict): 25000
len(ru_to_en_dict): 25000
len(ru_to_en_dict): 25000
len(ru_to_en_dict): 26000
len(ru_to_en_dict): 26000
len(ru_to_en_dict): 26000
len(ru_to_en_dict): 26000
len(ru_to_en_dict): 26000
len(ru_to_en_dict): 26000
len(ru_to_en_dict): 26000
len(ru_to_en_dict): 26000
len(ru_to_en_dict): 26000
len(ru_to_en_dict): 28000
len(ru_to_en_dict): 28000
len(ru_to_en_dict): 28000
len(ru_to_en_dict): 28000
len(ru_to_en_dict): 28000
len(ru_to_en_dict): 28000
len(ru_to_en_dict): 28000
len(ru_to_en_dict): 28000


  0%|          | 0/20014 [00:00<?, ?it/s]

len(ru_to_en_dict): 29000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 30000
len(ru_to_en_dict): 31000
len(ru_to_en_dict): 31000
len(ru_to_en_dict): 31000
len(ru_to_en_dict): 31000
len(ru_to_en_dict): 32000
len(ru_to_en_dict): 32000
len(ru_to_en_dict): 32000
len(ru_to_en_dict): 32000
len(ru_to_en_dict): 32000
len(ru_to_en_dict): 32000
len(ru_to_en_dict): 32000
len(ru_to_en_dict): 32000
len(ru_to_en_dict): 32000


  0%|          | 0/20646 [00:00<?, ?it/s]

len(ru_to_en_dict): 34000
len(ru_to_en_dict): 34000
len(ru_to_en_dict): 34000
len(ru_to_en_dict): 34000
len(ru_to_en_dict): 34000
len(ru_to_en_dict): 34000
len(ru_to_en_dict): 34000


  0%|          | 0/21004 [00:00<?, ?it/s]

(296988, 5) (321719, 5) (316871, 5) (301547, 5) (325032, 5)
Wall time: 4h 23min 36s


In [ ]:
%%time

emb_ru_google = pd.read_pickle('emb_ru_google_rubert-base-cased.pkl')
emb_ru_sarn = pd.read_pickle('emb_ru_sarn_rubert-base-cased.pkl')
emb_ru_kurl = pd.read_pickle('emb_ru_kurl_rubert-base-cased.pkl')
# emb_ru_arak = pd.read_pickle('emb_ru_arak_rubert-base-cased.pkl')
emb_en = pd.read_pickle('emb_en.pkl')

emb_ru_list = [emb_ru_sarn, emb_ru_kurl, emb_ru_google]

print(emb_ru_google.shape, emb_ru_sarn.shape,emb_ru_kurl.shape,  emb_en.shape)

(296988, 5) (321719, 5) (316871, 5) (325032, 5)
Wall time: 13.1 s


## Нам нужно получить "анкерные" слова -->

In [ ]:
class AbsoluteOrientation:
    def __init__(self, R=None, Y_mean=None):
        self.R = R
        self.Y_mean = Y_mean
    def fit(self, X, Y):
        X = np.stack(X)
        Y = np.stack(Y)
        # orthogonal, generic, rotational
        self.R = orthogonal(X, Y, ).t
        return self
    def predict(self, X):
        X = np.stack(X)
        return list(np.dot(X, self.R))

In [ ]:
def f_train(emb_ru, emb_en):

    def get_empty_en(x):
    # --берем только слова с >1 буквами и цифрами
        try:
            tmp = re.findall('^[a-z0-9]{1,}$', x)
            return tmp[0]
        except:
            return ''
        
    def get_empty_ru(x):
    # --берем только слова с >1 буквами и цифрами
        try:
            tmp = re.findall('^[а-я0-9]{1,}$', x)
            return tmp[0]
        except:
            return ''
             
#     убираем неполные слова :   
    emb_ru = emb_ru[(emb_ru['token_lower'].shift(-1).str[0] != '#')&(emb_ru['token_lower'].str[0] != '#')]
#     убираем редкие слова : 
    most_frequent_tokens = emb_ru.groupby('token_en')['token_lower'].agg(lambda x:x.value_counts().index[0]).values
    emb_ru = emb_ru[emb_ru['token_lower'].isin(most_frequent_tokens )] 
#     убираем короткие слова  
    emb_ru = emb_ru[emb_ru['token_lower'].apply(get_empty_ru) != '']    
    emb_ru = emb_ru[emb_ru['token_en'].apply(get_empty_en) != '']

    def get_one_vector(x):
        tmp = np.array(x['word_embedding'].values.tolist())
        d = {}
        d['word_embedding'] = np.mean(tmp, axis=0)
        d['count'] = tmp.shape[0]
        return pd.Series(d, index=['word_embedding', 'count'])

    # too_rare  -- убираем слишком редкие слова
    too_rare = 10

    tmp_ru = emb_ru.groupby(['token_en'], as_index = True).apply(get_one_vector).reset_index()
    tmp_ru = tmp_ru[tmp_ru['count'] > too_rare]
    tmp_ru = tmp_ru[['token_en','word_embedding']]

    tmp_en = emb_en.groupby(['token_en'], as_index = True).apply(get_one_vector).reset_index()
    tmp_en = tmp_en[tmp_en['count'] > too_rare]
    tmp_en = tmp_en[['token_en','word_embedding']]

    embeddings_matched = tmp_ru.set_index('token_en').join(tmp_en.set_index('token_en'), \
                                      how = 'inner', lsuffix = '_ru', rsuffix = '_en')
    print(f'nof unique words matched: {embeddings_matched.shape[0]}')

    pcd = paired_cosine_distances(embeddings_matched.iloc[:, 0].values.tolist(), \
                                  embeddings_matched.iloc[:, 1].values.tolist())

    print(f'pcd-before alignment: {np.mean(pcd)}', end = ', ')

    ao = AbsoluteOrientation()

    ao.fit(embeddings_matched.iloc[:, 0], embeddings_matched.iloc[:, 1])

    embeddings_matched['word_embedding_ru'] = ao.predict(embeddings_matched.iloc[:, 0])
    
    pcd = paired_cosine_distances(embeddings_matched.iloc[:, 0].values.tolist(), embeddings_matched.iloc[:, 1].\
                                  values.tolist())
    print(f'pcd-after alignment: {np.mean(pcd)}')
    
    ix_max = np.argmax(pcd)
    ix_min = np.argmin(pcd)
    print(f'smallest cd = {pcd[ix_min]}, en_token = {embeddings_matched.index[ix_min]}', end = ', ')
    print(f'biggest cd = {pcd[ix_max]}, en_token = {embeddings_matched.index[ix_max]}')
    return ao, embeddings_matched

# Преобразовываем все слова:

In [ ]:
%%time

ao, embeddings_matched = f_train(pd.concat([emb_ru_sarn, emb_ru_kurl, emb_ru_google, ]), emb_en)

for i in range(len(emb_ru_list)):
    emb_ru_list[i]['word_embedding'] = ao.predict(emb_ru_list[i]['word_embedding'])

nof unique words matched: 1251
pcd-before alignment: 0.9967246055603027, pcd-after alignment: 0.27649641036987305
smallest cd = 0.14409250020980835, en_token = he, biggest cd = 0.5158604383468628, en_token = besides
Wall time: 19.8 s


In [ ]:
tmp= emb_ru_sarn
tmp = tmp[tmp.token_lower=='зеленый'].sample(1)
print(tmp.token_lower)

d = cosine_similarity(tmp.word_embedding.values[0].reshape(1, -1), emb_en.word_embedding.values.tolist())[0]
ix_mx = np.argmax(d)
 
print(d[ix_mx])
emb_en.iloc[[ix_mx]]

6193    зеленый
Name: token_lower, dtype: object
0.5860814


,token,token_lower,token_en,txt,word_embedding
6801,green,green,green,"She was\nalready dialling Myra, and they were soon discussing the green awning\nwith great enthusiasm.","[-0.012329235, 0.026654841, 0.020789495, 0.017201733, -0.04070474, 0.032074496, -0.021143457, 0.042171177, -0.032908995, 0.018147502, -0.008657807, -0.001499812, -0.017121714, -0.015354901, 0.018883329, -0.018751258, -0.014211274, -0.0070660403, -0.0007140327, -0.004839001, -0.0208506, 0.012041258, -0.008456515, 0.007113686, 0.007918781, -0.004700831, 0.0012893945, 0.0397399, -0.01573223, 0.04..."


## Считаем BERTScore:

In [ ]:
pairs= pd.read_excel('resorch.xlsx', names = \
                     ['en', 'sarn','sarn_score','kurl', 'kurl_score', 'google', 'google_score', 'sum'])
pairs = pairs.dropna()

pairs['sarn_score']  = pairs['sarn_score'].astype('int')
pairs['kurl_score']  = pairs['kurl_score'].astype('int')
pairs['google_score']  = pairs['google_score'].astype('int')
pairs['sum']  = pairs['sum'].astype('int')

pairs = pairs[(pairs['sum'] == 3)& (pairs['sarn_score'] * pairs['kurl_score'] * pairs['google_score'] == 0)]

# pairs = pairs.head(1)

pairs.shape

(58, 8)

In [ ]:
%%time

for ind, candidate in enumerate(['sarn', 'kurl', 'google']):
#     print('*'*100)
#     print(candidate)
    emb_ru = emb_ru_list[ind]
    for ix, row in pairs[['en', candidate]].iterrows():
#         print(row.en, '\n', row[candidate], )
        tmp_en = emb_en[emb_en.txt == row.en]
        tmp_ru = emb_ru[emb_ru.txt == row[candidate]]
        precision = []
        for _, row_ru in tmp_ru.iterrows():
#             print(row_ru.token_lower, end = ':')
            d = cosine_similarity(row_ru.word_embedding.reshape(1, -1), \
                                      tmp_en.word_embedding.values.tolist())[0]
#             print(d)
            ix_mx = np.argmax(d)
#             print(tmp_en.iloc[ix_mx].token_en,': ', d[ix_mx])
            precision.append(d[ix_mx])

        precision = np.mean(precision)

        recall = []
        for _, row_en in tmp_en.iterrows():
#             print(row_en.token_lower, end = ':')
            d = cosine_similarity(row_en.word_embedding.reshape(1, -1), \
                                      tmp_ru.word_embedding.values.tolist())[0]
#             print(d)
            ix_mx = np.argmax(d)
#             print(tmp_ru.iloc[ix_mx].token_lower,': ', d[ix_mx])
            recall.append(d[ix_mx])
        recall = np.mean(recall)

#         print(precision, recall)
        
        pairs.loc[ix, candidate +'_p'] = precision
        pairs.loc[ix, candidate +'_r'] = recall
        pairs.loc[ix, candidate +'_f'] = 2* precision*recall/(precision+recall)
        
pairs =pairs[['en', 'sarn', 'kurl', 'google', \
              'sarn_score', 'kurl_score', 'google_score', 'sarn_f', 'kurl_f', 'google_f']]

Wall time: 9.8 s


In [ ]:
def f_get_spearmanr(x):
    t1 = x[['sarn_score', 'kurl_score', 'google_score']]
    t2 = x[['sarn_f', 'kurl_f', 'google_f']]
    return stats.spearmanr(t1, t2).correlation

pairs['spearmanr'] = pairs.apply(f_get_spearmanr, axis=1)

In [ ]:
pairs['spearmanr'].mean()

-0.3620689655172414

In [ ]:
pairs.head()

,en,sarn,kurl,google,sarn_score,kurl_score,google_score,sarn_f,kurl_f,google_f,spearmanr
3,"He produced a plain white card from\nhis back pocket, opened the mail-slot, and dropped the card through.","Из заднего кармана он достал белую карточку, открыл щель для почты, просунул туда карточку и кончиками пальцев придержал крышку прорези, чтобы она не щелкнула, когда закрывалась.","Достав из заднего кармана брюк белую карточку, он сунул ее в почтовую щель на двери.","Он достал простую белую карточку из заднего кармана, открыл прорезь для почты и вытащил карту внутрь.",1,2,0,0.461813,0.477418,0.513289,-0.5
5,"While Alan was thinking, she had taken off the tie\nand dextrously unbuttoned the top two buttons of the white blouse\nunderneath.","Пока Алан был погружен в свои мысли, она успела снять галстук и ловко расстегнуть две верхние пуговки белой блузки под джемпером.","Теперь же платочек был развязан и Полли расстегивала две верхние пуговки белой блузки, надетой под джемпер.","Пока Алан думал, она сняла галстук и ловко расстегнула две верхние пуговицы белой блузки под ним .",2,1,0,0.457043,0.422092,0.480357,-0.5
7,He was\nwearing a blue tee-shirt-what the schoolkids called a muscle-shirt-and\nblue sweatpants with a white stripe on the side.,"На нем была голубая майка — ребятишки называют такие футболками, — и голубые спортивные штаны с белой полоской по бокам.","На нем были голубая футболка из тех, что мальчики называют борцовками, и синие спортивные брюки с белыми полосками по бокам.","На нем была синяя футболка - то, что школьники называли мускулистой рубашкой - и синие спортивные штаны с белой полосой по бокам.",1,2,0,0.449948,0.457202,0.454874,0.5
9,Pratt was a beefy young Baptist who usually wore blue\ntee-shirts and blue sweat-pants with a white stripe running down the\noutside of each leg.,Пратт был жирным молодым баптистом и обычно носил голубые майки и голубые тренировочные штаны (мокрые от пота) с белыми полосками по внешним швам.,"Пратт был крепкий телом молодой баптист, всегда ходил в голубых футболках и тренировочных штанах с белыми полосами вдоль обеих штанин снаружи.","Пратт был крепким молодым баптистом, который обычно носил синие футболки и синие спортивные штаны с белой полосой, идущей по внешней стороне каждой ноги.",0,1,2,0.471452,0.447446,0.489664,0.5
17,"She was\nalready dialling Myra, and they were soon discussing the green awning\nwith great enthusiasm.","Она уже набирала номер Майры, и сейчас они примутся с большим энтузиазмом обсуждать зеленый навес.",И минуту спустя они с превеликим энтузиазмом обсуждали зеленый навес.,"Она уже набирала номер Майры, и вскоре они с большим энтузиазмом обсуждали зеленый навес.",1,0,2,0.431667,0.441571,0.466551,0.5


In [ ]:
pairs['sarn_f'] = pairs['sarn_f'].map(lambda x:  rn.random())

pairs['kurl_f'] = pairs['kurl_f'].map(lambda x:  rn.random())

pairs['google_f'] = pairs['google_f'].map(lambda x:  rn.random())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(tmp_en.reshape(1, -1), tmp_ru.reshape(1, -1) )[0][0]

0.77203256

## Альтернативный подход: cчитаем cosine_similarity только для цветообозначений

In [ ]:
%%time

for ind, candidate in enumerate(['sarn', 'kurl', 'google']):
    print('*'*100)
    print(candidate)
    emb_ru = emb_ru_list[ind]
    for ix, row in pairs[['en', candidate]].iterrows():
#         print(row.en, '\n', row[candidate], )

        tmp_en = emb_en[(emb_en.txt == row.en)& emb_en.token_en.isin(phrases_en['en'])].head(1)
        tmp_ru = emb_ru[(emb_ru.txt == row[candidate])& emb_ru.token_en.isin(phrases_en['en'])].head(1)
        
        if tmp_en.token_en.values[0] != tmp_ru.token_en.values[0]:
            print(tmp_en.token_en.values[0], ':', tmp_ru.token_en.values[0]) 
    
        tmp_en = tmp_en['word_embedding'].values[0]
        
        tmp_ru = tmp_ru['word_embedding'].values[0]
        
        
        cs = cosine_similarity(tmp_en.reshape(1, -1), tmp_ru.reshape(1, -1) )[0][0]
        
        pairs.loc[ix, candidate +'_cs'] = cs
        
pairs = pairs[['en', 'sarn', 'kurl', 'google', \
              'sarn_score', 'kurl_score', 'google_score', 'sarn_cs', 'kurl_cs', 'google_cs']]

****************************************************************************************************
sarn
black : pink
gray : grey
gray : brown
****************************************************************************************************
kurl
black : pink
gray : grey
gray : grey
black : white
****************************************************************************************************
google
white : black
gray : green
Wall time: 11.8 s


In [ ]:
def f_get_spearmanr(x):
    t1 = x[['sarn_score', 'kurl_score', 'google_score']]
    t2 = x[['sarn_cs', 'kurl_cs', 'google_cs']]
    return stats.spearmanr(t1, t2).correlation

pairs['spearmanr'] = pairs.apply(f_get_spearmanr, axis=1)

In [ ]:
pairs['spearmanr'].mean()

-0.07758620689655173